In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/railway-track-fault-detection/Railway Track fault Detection Updated/Validation/Defective/IMG_20201211_121249.jpg
/kaggle/input/railway-track-fault-detection/Railway Track fault Detection Updated/Validation/Defective/IMG_20201211_121703.jpg
/kaggle/input/railway-track-fault-detection/Railway Track fault Detection Updated/Validation/Defective/IMG_20201211_121541.jpg
/kaggle/input/railway-track-fault-detection/Railway Track fault Detection Updated/Validation/Defective/IMG_20201114_102932.jpg
/kaggle/input/railway-track-fault-detection/Railway Track fault Detection Updated/Validation/Defective/IMG_20201114_102159.jpg
/kaggle/input/railway-track-fault-detection/Railway Track fault Detection Updated/Validation/Defective/IMG_20201211_121524.jpg
/kaggle/input/railway-track-fault-detection/Railway Track fault Detection Updated/Validation/Defective/IMG_20201211_121655.jpg
/kaggle/input/railway-track-fault-detection/Railway Track fault Detection Updated/Validation/Defective/IMG_2020

In [2]:
import tensorflow as tf
from tensorflow.keras import layers,regularizers
from tensorflow import keras
import cv2 
from tqdm import tqdm

In [3]:
for dirname,_,_ in os.walk("/kaggle/input"): 
    print(dirname)

/kaggle/input
/kaggle/input/railway-track-fault-detection
/kaggle/input/railway-track-fault-detection/Railway Track fault Detection Updated
/kaggle/input/railway-track-fault-detection/Railway Track fault Detection Updated/Validation
/kaggle/input/railway-track-fault-detection/Railway Track fault Detection Updated/Validation/Defective
/kaggle/input/railway-track-fault-detection/Railway Track fault Detection Updated/Validation/Non defective
/kaggle/input/railway-track-fault-detection/Railway Track fault Detection Updated/Test
/kaggle/input/railway-track-fault-detection/Railway Track fault Detection Updated/Test/Defective
/kaggle/input/railway-track-fault-detection/Railway Track fault Detection Updated/Test/Non defective
/kaggle/input/railway-track-fault-detection/Railway Track fault Detection Updated/Train
/kaggle/input/railway-track-fault-detection/Railway Track fault Detection Updated/Train/Defective
/kaggle/input/railway-track-fault-detection/Railway Track fault Detection Updated/Trai

In [4]:
validationDataPath="/kaggle/input/railway-track-fault-detection/Railway Track fault Detection Updated/Validation"
trainDataPath="/kaggle/input/railway-track-fault-detection/Railway Track fault Detection Updated/Train"
testDataPath="/kaggle/input/railway-track-fault-detection/Railway Track fault Detection Updated/Test"

In [5]:
from keras import Sequential
from keras.layers import Conv2D,Input,Dense,MaxPooling2D,Flatten,BatchNormalization,Dropout
from keras.preprocessing import image_dataset_from_directory

In [6]:
trainData=tf.keras.preprocessing.image_dataset_from_directory(
    trainDataPath,
    batch_size=32,
    image_size=(224,224),
    shuffle=True
)

Found 299 files belonging to 2 classes.


In [7]:
testData=image_dataset_from_directory(
    testDataPath,
    batch_size=32,
    image_size=(224,224),
    shuffle=True
)

Found 22 files belonging to 2 classes.


In [8]:
validationData=image_dataset_from_directory(
    validationDataPath,
    batch_size=32,
    image_size=(224,224),
    shuffle=True
)

Found 62 files belonging to 2 classes.


In [9]:
trainData.class_names

['Defective', 'Non defective']

In [10]:
datagen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)
testScaledData=datagen.flow_from_directory(
    testDataPath,
    batch_size=32,
    target_size=(224,224),
    shuffle=True,
    class_mode='sparse'
)

trainScaledData=datagen.flow_from_directory(
    trainDataPath,
    batch_size=32,
    target_size=(224,224),
    shuffle=True,
    class_mode='sparse'
)

validationScaledData=datagen.flow_from_directory(
    validationDataPath,
    batch_size=32,
    target_size=(224,224),
    shuffle=True,
    class_mode='sparse'
)

Found 22 images belonging to 2 classes.
Found 299 images belonging to 2 classes.
Found 62 images belonging to 2 classes.


In [11]:
for img,label in testScaledData: 
    print(label)
    break

[1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1.]


In [12]:
model=Sequential()

In [13]:
model.add(Conv2D(32,(3,3),input_shape=trainScaledData.image_shape,activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Flatten())

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
model.add(Dropout(0.2))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(trainScaledData.num_classes,activation='sigmoid'))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 21632)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 21632)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     5,538,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,590,594 (21.33 MB)

 Trainable params: 5,590,594 (21.33 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [16]:
batch_size=32
model.fit(
    trainScaledData,
    steps_per_epoch = len(trainScaledData) // batch_size,
    validation_data=validationScaledData,
    validation_steps = len(validationScaledData) // batch_size,
    epochs=10
)

Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


10/10 ━━━━━━━━━━━━━━━━━━━━ 81s 4s/step - accuracy: 0.5266 - loss: 0.8665 - val_accuracy: 0.5000 - val_loss: 0.6923
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 56s 3s/step - accuracy: 0.5765 - loss: 0.6789 - val_accuracy: 0.5645 - val_loss: 0.6481
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 57s 3s/step - accuracy: 0.6491 - loss: 0.6355 - val_accuracy: 0.8387 - val_loss: 0.5714
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - accuracy: 0.7154 - loss: 0.5947 - val_accuracy: 0.5161 - val_loss: 0.7340
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 55s 3s/step - accuracy: 0.6696 - loss: 0.5554 - val_accuracy: 0.8387 - val_loss: 0.5064
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 55s 3s/step - accuracy: 0.7801 - loss: 0.4729 - val_accuracy: 0.8871 - val_loss: 0.4030
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - accuracy: 0.8528 - loss: 0.3545 - val_accuracy: 0.6129 - val_loss: 0.6526
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - accuracy: 0.8698 - loss: 0.3120 - val_accuracy: 0.8065 - val_loss: 0.3945
Epo

In [17]:
model.save("RailTrackFaultDetection.h5")

In [18]:
loss,acc=model.evaluate(trainScaledData)
print("Loss on train data: ",loss)
print("Accuracy on train data: ",acc)

10/10 ━━━━━━━━━━━━━━━━━━━━ 39s 4s/step - accuracy: 0.9755 - loss: 0.1811
Loss on train data:  0.18516959249973297
Accuracy on train data:  0.9698996543884277


In [19]:
loss1,acc1=model.evaluate(validationScaledData)
print("Loss on validation data: ",loss1)
print("Accuracy on validation data: ",acc1)

2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 5s/step - accuracy: 0.7026 - loss: 0.5597
Loss on validation data:  0.5506008863449097
Accuracy on validation data:  0.725806474685669


In [20]:
loss2,acc2=model.evaluate(testScaledData)
print("Loss on test data: ",loss2)
print("Accuracy on test data: ",acc2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.6818 - loss: 0.6831
Loss on test data:  0.6830692291259766
Accuracy on test data:  0.6818181872367859


In [21]:
dict_class={'Label': trainData.class_names}
dict_class

{'Label': ['Defective', 'Non defective']}

In [22]:
import json
with open("class_labels.json","w") as file: 
    json.dump(dict_class,file)

In [23]:
with open("class_labels.json", "r") as file:
    class_labels = json.load(file)

In [24]:
loadedModel = tf.keras.models.load_model("./RailTrackFaultDetection.h5")

def loadImg(imgPath): 
    img = tf.keras.preprocessing.image.load_img(imgPath, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array=img_array/255.0
    return img_array

In [25]:
testFolder="/kaggle/input/railway-track-fault-detection/Railway Track fault Detection Updated/Test"
def predictImagesInFolder(testFolder): 
    for subdir,dirs,files in os.walk(testFolder): 
        if subdir==testFolder: 
            continue
        trueLabel=os.path.basename(subdir)
        for file in files: 
            img_path=os.path.join(subdir,file)
            img=loadImg(img_path)
            predictions=loadedModel.predict(img)
            predicted_class=np.argmax(predictions,axis=1)
            print(f"Predicted class: {predicted_class[0]} File: {file}")
predictImagesInFolder(testFolder)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
Predicted class: 0 File: IMG_20201114_100209.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predicted class: 0 File: IMG_20201114_102203.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Predicted class: 0 File: IMG_20201114_101124.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predicted class: 0 File: IMG_20201114_102819.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predicted class: 1 File: IMG_20201211_121712_1.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predicted class: 1 File: IMG_20201211_121713.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Predicted class: 1 File: IMG_20201114_103110.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Predicted class: 0 File: IMG_20201114_102222.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predicted class: 1 File: IMG_20201211_121712.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Predicted class: 0 File: IMG_20201114_100159.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Predicted class: 0 File: IMG_20201114_101200.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Pr